In [ ]:
!rm -rf test
!rm -rf valid
# mappa teljes törlése

In [ ]:
!pip install -q tflite-model-maker

In [ ]:
import os

import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
#from tflite_model_maker import image_classifier
from tflite_model_maker import object_detector
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# **Kaggle**

In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
#!kaggle datasets download -d gpiosenka/100-bird-species
!kaggle datasets download -d vvvalter/chess-pieces
#!unzip 100-bird-species.zip

100% 1.05G/1.05G [00:11<00:00, 136MB/s]
100% 1.05G/1.05G [00:11<00:00, 95.2MB/s]


# **Roboflow**

In [ ]:
!curl -L "https://public.roboflow.com/ds/dK7QBUyf2y?key=tnz5phNGBO" > chess-pieces.zip;  # csv


#!curl -L "https://public.roboflow.com/ds/yeRc0xFJsl?key=Aj33LH3RLO" > chess-pieces.zip;  # pascal voc

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   894  100   894    0     0   2013      0 --:--:-- --:--:-- --:--:--  2013
100 16.8M  100 16.8M    0     0  21.0M      0 --:--:-- --:--:-- --:--:-- 82.7M


In [ ]:
import zipfile
#with zipfile.ZipFile('100-bird-species.zip', 'r') as zip_ref:
with zipfile.ZipFile('chess-pieces.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

In [ ]:
nrows = 4
ncols = 4
#imagenames = os.listdir(os.path.join("train", "ABBOTTS BABBLER"))
imagenames = os.listdir(os.path.join("chess-pieces", "black bishop"))

fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

#next_images = [os.path.join(os.path.join("train", "ABBOTTS BABBLER"), fname) for fname in imagenames[:int(ncols*nrows)]]
next_images = [os.path.join(os.path.join("chess-pieces", "black bishop"), fname) for fname in imagenames[:int(ncols*nrows)]]

for i, img_path in enumerate(next_images):
    sp = plt.subplot(nrows, ncols, i + 1)
    img = mpimg.imread(img_path)
    plt.imshow(img)

plt.show()

In [ ]:
spec = model_spec.get('efficientdet_lite4')

#data = DataLoader.from_folder('train')
#data = DataLoader.from_folder('chess-pieces')
#data = object_detector.DataLoader.from_csv(os.path.join("train", "_annotations.csv"))

train_data, validation_data, test_data = object_detector.DataLoader.from_csv("annotations.csv")

#train_data, test_data = data.split(0.9)

#train_data, rest_data = data.split(0.8)
#validation_data, test_data = rest_data.split(0.5)

INFO:tensorflow:Cache will be stored in /tmp/tmpp7fm6n0l with prefix filename train_d155c6d48da36cf8f5f39f5ad77b0b9d. Cache_prefix is /tmp/tmpp7fm6n0l/train_d155c6d48da36cf8f5f39f5ad77b0b9d
INFO:tensorflow:Cache will be stored in /tmp/tmp_gw5jtlz with prefix filename val_d155c6d48da36cf8f5f39f5ad77b0b9d. Cache_prefix is /tmp/tmp_gw5jtlz/val_d155c6d48da36cf8f5f39f5ad77b0b9d
INFO:tensorflow:Cache will be stored in /tmp/tmp2ibeuj7v with prefix filename test_d155c6d48da36cf8f5f39f5ad77b0b9d. Cache_prefix is /tmp/tmp2ibeuj7v/test_d155c6d48da36cf8f5f39f5ad77b0b9d
INFO:tensorflow:On image 0
INFO:tensorflow:On image 100
INFO:tensorflow:On image 200
INFO:tensorflow:On image 300
INFO:tensorflow:On image 0
INFO:tensorflow:On image 100
INFO:tensorflow:On image 0
INFO:tensorflow:On image 100


In [ ]:
plt.figure(figsize=(20,20))
for i, (image, label) in enumerate(data.gen_dataset().unbatch().take(25)):
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image.numpy(), cmap=plt.cm.gray)
  plt.xlabel(data.index_to_label[label.numpy()])
plt.show()

In [ ]:
#model = image_classifier.create(train_data, epochs=10)
#model = image_classifier.create(train_data, validation_data=validation_data)
model = object_detector.create(train_data, model_spec=spec, batch_size=8, train_whole_model=False, validation_data=validation_data)

INFO:tensorflow:Retraining the models...
Epoch 1/50
47/47 [==============================] - 128s 2s/step - det_loss: 1.6360 - cls_loss: 1.1719 - box_loss: 0.0093 - reg_l2_loss: 0.0090 - loss: 1.6449 - learning_rate: 0.0090 - gradient_norm: 0.6900 - val_det_loss: 1.2006 - val_cls_loss: 0.9782 - val_box_loss: 0.0044 - val_reg_l2_loss: 0.0090 - val_loss: 1.2096
Epoch 2/50
47/47 [==============================] - 81s 2s/step - det_loss: 1.2135 - cls_loss: 0.9067 - box_loss: 0.0061 - reg_l2_loss: 0.0090 - loss: 1.2225 - learning_rate: 0.0100 - gradient_norm: 0.9602 - val_det_loss: 0.8606 - val_cls_loss: 0.6936 - val_box_loss: 0.0033 - val_reg_l2_loss: 0.0090 - val_loss: 0.8696
Epoch 3/50
47/47 [==============================] - 80s 2s/step - det_loss: 0.9815 - cls_loss: 0.7514 - box_loss: 0.0046 - reg_l2_loss: 0.0090 - loss: 0.9905 - learning_rate: 0.0099 - gradient_norm: 1.2355 - val_det_loss: 0.7091 - val_cls_loss: 0.5673 - val_box_loss: 0.0028 - val_reg_l2_loss: 0.0090 - val_loss: 0.718

In [ ]:
#loss, accuracy = model.evaluate(test_data)

model.evaluate(test_data)

2/2 [==============================] - 58s 22s/step

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
Converting ndarray to lists...
(10500, 7)
0/10500
DONE (t=0.04s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.14s).
Accumulating evaluation results...
DONE (t=0.28s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.535
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.825
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.651
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.471
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.539
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.427
 Average Recall     (AR) @[ IoU=0.50:0.95

{'AP': 0.5347021,
 'AP50': 0.8254617,
 'AP75': 0.6514567,
 'AP_/bishop': -1.0,
 'AP_/black-bishop': 0.45304334,
 'AP_/black-king': 0.64458585,
 'AP_/black-knight': 0.5375624,
 'AP_/black-pawn': 0.6156935,
 'AP_/black-queen': 0.39348987,
 'AP_/black-rook': 0.56167245,
 'AP_/white-bishop': 0.45994717,
 'AP_/white-king': 0.57714045,
 'AP_/white-knight': 0.5525292,
 'AP_/white-pawn': 0.62518066,
 'AP_/white-queen': 0.43317562,
 'AP_/white-rook': 0.5624048,
 'APl': -1.0,
 'APm': 0.5393473,
 'APs': 0.4713641,
 'ARl': -1.0,
 'ARm': 0.68043715,
 'ARmax1': 0.42663592,
 'ARmax10': 0.673957,
 'ARmax100': 0.6769308,
 'ARs': 0.60651886}

In [ ]:
model.summary()

In [ ]:
def get_label_color(val1, val2):
  if val1 == val2:
    return 'black'
  else:
    return 'red'

plt.figure(figsize=(25, 20))
predicts = model.predict_top_k(test_data)
for i, (image, label) in enumerate(test_data.gen_dataset().unbatch().take(25)):
  ax = plt.subplot(5, 5, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image.numpy(), cmap=plt.cm.gray)

  predict_label = predicts[i][0][0]
  color = get_label_color(predict_label,
                          test_data.index_to_label[label.numpy()])
  ax.xaxis.label.set_color(color)
  plt.xlabel(predict_label)
plt.show()

In [ ]:
model.export(export_dir='.', tflite_filename='chess-piece-detector.tflite')

INFO:tensorflow:Assets written to: /tmp/tmp21s6yrlp/assets


INFO:tensorflow:Assets written to: /tmp/tmp21s6yrlp/assets


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Saving labels in /tmp/tmpsf9mbk9m/labelmap.txt.


INFO:tensorflow:Saving labels in /tmp/tmpsf9mbk9m/labelmap.txt.


INFO:tensorflow:TensorFlow Lite model exported successfully: ./chess-piece-detector.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: ./chess-piece-detector.tflite


In [ ]:
model.evaluate_tflite('chess-piece-detector.tflite', test_data)

 68/105 [==================>...........] - ETA: 32:37